In [24]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
# 数据加载
Train_data = pd.read_csv('./train_data.csv', index_col=0)
Test_data = pd.read_csv('./test_data.csv', index_col=0)
Test_data.drop('Attrition', axis=1, inplace=True)
x_train, x_test, y_train, y_test = train_test_split(Train_data.drop('Attrition', axis=1), Train_data['Attrition'], test_size=0.2)

In [25]:
param = {'boosting_type':'gbdt',
                         'objective' : 'binary:logistic', #
                         'eval_metric' : 'auc',
                         'eta' : 0.01,
                         'max_depth' : 15,
                         'colsample_bytree':0.8,
                         'subsample': 0.9,
                         'subsample_freq': 8,
                         'alpha': 0.6,
                         'lambda': 0,}

In [26]:
train_data = xgb.DMatrix(x_train, label=y_train)
valid_data = xgb.DMatrix(x_test, label=y_test)
test_data = xgb.DMatrix(Test_data)


In [27]:
model = xgb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round=10000, 
                 early_stopping_rounds=200, verbose_eval=25)
predict = model.predict(test_data)

[0]	train-auc:0.860529	valid-auc:0.71717
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 200 rounds.
[25]	train-auc:0.975945	valid-auc:0.778657
[50]	train-auc:0.98581	valid-auc:0.787064
[75]	train-auc:0.99209	valid-auc:0.794919
[100]	train-auc:0.995368	valid-auc:0.792096
[125]	train-auc:0.996679	valid-auc:0.800442
[150]	train-auc:0.99774	valid-auc:0.803019
[175]	train-auc:0.99843	valid-auc:0.804001
[200]	train-auc:0.998922	valid-auc:0.806578
[225]	train-auc:0.999388	valid-auc:0.806087
[250]	train-auc:0.999698	valid-auc:0.806087
[275]	train-auc:0.999802	valid-auc:0.807806
[300]	train-auc:0.999922	valid-auc:0.809647
[325]	train-auc:0.99994	valid-auc:0.810997
[350]	train-auc:0.999965	valid-auc:0.813574
[375]	train-auc:0.999983	valid-auc:0.814188
[400]	train-auc:0.999983	valid-auc:0.81566
[425]	train-auc:1	valid-auc:0.817133
[450]	train-auc:1	valid-auc:0.817256
[475]	train-auc:1	valid-auc:0.817379
[500]	tra

In [31]:
temp = pd.read_csv('./test.csv')['user_id']
# print(temp)
submit = pd.DataFrame(temp)
submit['Attrition'] = predict
submit['Attrition'] = submit['Attrition'].map(lambda x:1 if x>=0.5 else 0)
print(submit)
submit.to_csv('submit_xgb.csv', index=None)

     user_id  Attrition
0        442          0
1       1091          0
2        981          0
3        785          0
4       1332          1
..       ...        ...
289     1439          0
290      481          0
291      124          1
292      198          0
293     1229          0

[294 rows x 2 columns]
